In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine

In [2]:
#read csvs
wv2_w_v1 = pd.read_csv('CSV_files/tables_W/wv2_w.csv')
wv3_w_v1 = pd.read_csv('CSV_files/tables_W/wv3_w.csv')
wv4_w_v1 = pd.read_csv('CSV_files/tables_W/wv4_w.csv')
wv5_w_v1 = pd.read_csv('CSV_files/tables_W/wv5_w.csv')
wv6_w_v1 = pd.read_csv('CSV_files/tables_W/wv6_w.csv')

In [3]:
# Turn column names into lists
wv2_w_v1_columns = list(wv2_w_v1.columns)
wv3_w_v1_columns = list(wv3_w_v1.columns)
wv4_w_v1_columns = list(wv4_w_v1.columns)
wv5_w_v1_columns = list(wv5_w_v1.columns)
wv6_w_v1_columns = list(wv6_w_v1.columns)

In [4]:
wv2_w_v1.head()

,caseid_new,caseid_wave,qualified?,w2_broke_up,w2_xmarry,w2_deceased,w2_panelstat,w2_donotcontact,w2_assigned,w2_f1complete,...,w2_q1,w2_q5,w2_q2,w2_q3,w2_q4,w2_q6,w2_q7,w2_q8,w2_q9,w2_q10
0,22526,22526_2,qualified,still together,partnered,not deceased,withdrawn kn panelist,all other cases,assigned to survey,completed followup survey,...,NaN,yes,NaN,NaN,NaN,yes,"no, did not marry [partner]","no, we have not gotten a domestic partnership ...",NaN,NaN
1,23286,23286_2,qualified,still together,married,not deceased,withdrawn kn panelist,all other cases,assigned to survey,completed followup survey,...,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26315,26315_2,qualified,broke up,partnered,not deceased,withdrawn kn panelist,all other cases,assigned to survey,completed followup survey,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,we broke up,i wanted to break up more
3,28536,28536_2,qualified,still together,partnered,not deceased,active kn panelist,all other cases,assigned to survey,completed followup survey,...,NaN,yes,NaN,NaN,NaN,yes,"no, did not marry [partner]","no, we have not gotten a domestic partnership ...",NaN,NaN
4,29584,29584_2,qualified,still together,married,not deceased,active kn panelist,all other cases,assigned to survey,completed followup survey,...,yes,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# clean df1 (waves 1-3)

In [5]:
wv2_w_v2 = wv2_w_v1[[
'caseid_new',
'caseid_wave', #newly added
'qualified?', #newly added                                        
'w2_f1complete', # completed wave 2 followup study
'w2_HCMST_interview_fin_yrmo', #interview year/month
'w2_broke_up', # for both married and partnered (options: still together, broke up, partner passed away)
'w2_q7',  # in the past year, have you and partner gotten married?
# 'w2_assigned',
# 'w2_xmarry',           
# 'w2_deceased',
# 'w2_panelstat',
# 'w2_donotcontact',
# 'w2_xss', # same sex partner
# 'w2_q1', # are you still married to your partner?
# 'w2_q5',  # are you still in a romantic relationship with partner?
# 'w2_q2',  # are you currently living with partner?
# 'w2_q3', # how did your marriage end? (only for breakups)
# 'w2_q4',  # who wanted the divorce mroe?
# 'w2_q6',  # are you currently living with your partner (non-married people)
# 'w2_q8', # in past 12 months, have you gotten into formal domestic partnership?
# 'w2_q9', # how did your relationship end? (non-married people)
# 'w2_q10' # between you and partner, who wanted to break up? (non-married people)
 #'w2_days_elapsed'
]].copy()

In [6]:
wv2_w_v2_columns = list(wv2_w_v2)
print(wv2_w_v2.shape)
wv2_w_v2.head()

(3009, 7)


,caseid_new,caseid_wave,qualified?,w2_f1complete,w2_HCMST_interview_fin_yrmo,w2_broke_up,w2_q7
0,22526,22526_2,qualified,completed followup survey,201003.0,still together,"no, did not marry [partner]"
1,23286,23286_2,qualified,completed followup survey,201003.0,still together,NaN
2,26315,26315_2,qualified,completed followup survey,201003.0,broke up,NaN
3,28536,28536_2,qualified,completed followup survey,201003.0,still together,"no, did not marry [partner]"
4,29584,29584_2,qualified,completed followup survey,201003.0,still together,NaN


In [7]:
print(f'col Length of wv2_w_v2_columns: {len(wv2_w_v2_columns)}')
wv2_w_v2_columns

col Length of wv2_w_v2_columns: 7


['caseid_new',
 'caseid_wave',
 'qualified?',
 'w2_f1complete',
 'w2_HCMST_interview_fin_yrmo',
 'w2_broke_up',
 'w2_q7']

# WAVE 3

In [8]:
wv3_w_v2 = wv3_w_v1[[
'caseid_new', 
'caseid_wave', #newly added
'qualified?', #newly added                                     
'w3_complete', # complete wave 3 survey? 
'w3_HCMST_interview_fin_yrmo', # year/month of 3rd wave interview   
'w3_broke_up',
'w3_q7', # have you and partner gotten married in last year       
'w3_mbtiming_year',### year of breakup (married)
'w3_mbtiming_month',### month of breakup (married)
'w3_nonmbtiming_year', ### year of breakup (non-married)
'w3_nonmbtiming_month',### month of breakup (non-married)   
 
#  'w3_xmarry',
#  'w3_xdeceased', #couple broke up or partner deceased at wave 3
#  'w3_xpartnered', # is subject partnered
#  'w3_xss',
    
#  'w3_xlast', # whether responded was last contacted 1 or 2 years ago
#  'w3_xyear', # year of last contact
#   'w3_xmonth', # month of last contact

#  'w3_xqualified', # is subject qualified for wave 3
#  'w3_status',
#  'w3_days_elapsed',
#  'w3_xtype', # same-sex or heterosexual?
#  'w3_q1', # still married?
#  'w3_q2', # still living together?
#  'w3_q3', # how did marriage end?                  
#  'w3_q4', # who wanted marriage to end more?
#  'w3_q5', # still in romantic relationship with partner?
#  'w3_q6', # currently living with partner?
#  'w3_q8', # did you get into domestic partnership in last year?
#   'w3_q9', # how did relationship end
#   'w3_q10', # who wanted to break up more
]].copy()

In [9]:
wv3_w_v2_columns = list(wv3_w_v2)
print(wv3_w_v2.shape)
wv3_w_v2

(3009, 11)


,caseid_new,caseid_wave,qualified?,w3_complete,w3_HCMST_interview_fin_yrmo,w3_broke_up,w3_q7,w3_mbtiming_year,w3_mbtiming_month,w3_nonmbtiming_year,w3_nonmbtiming_month
0,22526,22526_3,qualified,yes,201104.0,still together,"no, did not marry [xNameP]",NaN,NaN,NaN,NaN
1,23286,23286_3,qualified,yes,201104.0,still together,NaN,NaN,NaN,NaN,NaN
2,26315,26315_3,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28536,28536_3,qualified,yes,201104.0,still together,"no, did not marry [xNameP]",NaN,NaN,NaN,NaN
4,29584,29584_3,qualified,yes,201104.0,still together,NaN,NaN,NaN,NaN,NaN
5,31456,31456_3,qualified,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,32656,32656_3,qualified,yes,201106.0,still together,"no, did not marry [xNameP]",NaN,NaN,NaN,NaN
7,33536,33536_3,qualified,yes,201104.0,still together,"no, did not marry [xNameP]",NaN,NaN,NaN,NaN
8,34341,34341_3,qualified,yes,201104.0,still together,NaN,NaN,NaN,NaN,NaN
9,35653,35653_3,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print(f'col Length of wv3_w_v2_columns: {len(wv3_w_v2_columns)}')
wv3_w_v2_columns

col Length of wv3_w_v2_columns: 11


['caseid_new',
 'caseid_wave',
 'qualified?',
 'w3_complete',
 'w3_HCMST_interview_fin_yrmo',
 'w3_broke_up',
 'w3_q7',
 'w3_mbtiming_year',
 'w3_mbtiming_month',
 'w3_nonmbtiming_year',
 'w3_nonmbtiming_month']

# Wave 4

In [11]:
wv4_w_v2 = wv4_w_v1[[
'caseid_new',
'caseid_wave', #newly added
'qualified?', #newly added
'w4_complete',
'w4_HCMST_interview_fin_yrmo', # year and month subject responded to wv4    
'w4_broke_up',
'w4_q7', # in the past few years have you gotten married?
'w4_mbtiming_year', # when did you get divorced/separated?
'w4_mbtiming_month', # month of divorce/separation
'w4_nonmbtiming_year',
'w4_nonmbtiming_month',
'w4_mar_year', # year couple got married
'w4_mar_month', # month got married
'relationship_quality_w4', # quality of relationship converted to number ranking
'w4_attractive', # how attractive is partner?
'w4_attractive_partner',
# 'w234_combo_breakup',    
# 'w4_xqualified',
# 'w4_status',
#  'w4_source',
#  'w4_duration',
# 'w4_xtype', # same sex vs hetero
# 'w4_xyear', # year of last contact with subject
# # 'w4_xmonth',
# # 'w4_xmarry',
# 'w4_xlast', # how many years since last contact
# # 'w4_xwave1_month',
# # 'w4_xcivil',
# 'w4_xss', # same sex, not domestic partnership
# 'w4_q1', # still married to partner?
# 'w4_q2', # currently living with partner?
# 'w4_quality', # quality of relationship with partner?
# 'w4_q3', # how did marriage end?
# 'w4_q4', # how did marriage end? (breakup only)
# 'w4_q5', # are you still in a romantic relationship with partner? (non-married)
# 'w4_q6', # still living together?
# 'w4_civ_year', # year couple got into domestic partnership or civil union
# 'w4_q8_a', # in the past yeras, gotten into formal domestic partnership?
# 'w4_q8_b',
# 'w4_q9', # hwo did relationship end?
# 'w4_q10', # who wanted to break up? (non-married breakup)
]].copy()

In [12]:
wv4_w_v2_columns = list(wv4_w_v2)
print(wv4_w_v2.shape)
wv4_w_v2

(4002, 16)


,caseid_new,caseid_wave,qualified?,w4_complete,w4_HCMST_interview_fin_yrmo,w4_broke_up,w4_q7,w4_mbtiming_year,w4_mbtiming_month,w4_nonmbtiming_year,w4_nonmbtiming_month,w4_mar_year,w4_mar_month,relationship_quality_w4,w4_attractive,w4_attractive_partner
0,22526,22526_4,qualified,Yes,201303.0,still together,"No, did not marry [xnamep]",NaN,NaN,NaN,NaN,NaN,NaN,4.0,moderately attractive,very attractive
1,23286,23286_4,qualified,Yes,201303.0,still together,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,very attractive,very attractive
2,25495,25495_4,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26315,26315_4,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27355,27355_4,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,27695,27695_4,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,28536,28536_4,qualified,Yes,201303.0,still together,"No, did not marry [xnamep]",NaN,NaN,NaN,NaN,NaN,NaN,3.0,moderately attractive,moderately attractive
7,29584,29584_4,qualified,Yes,201303.0,still together,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,moderately attractive,moderately attractive
8,30393,30393_4,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,31456,31456_4,qualified,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Wave 5

In [13]:
wv5_w_v2 = wv5_w_v1[[
'caseid_new',
'caseid_wave', #newly added
'qualified?', #newly added   
'w5_complete', # interview completion flag
'w5_HCMST_interview_fin_yrmo',# year and month participant responded to wave 5  
'w5_broke_up',
'w5_q7', # in the past years (since last interview), have you gotten marreid?    
'w5_mbtiming_month', #IMPORTANT what month did you break up with partner?
'w5_mbtiming_year',#IMPORTANT what year did you break up with partner?   
'w5_nonmbtiming_year', # year of breakup (non-marreid)
'w5_nonmbtiming_month', # month of breakup (non-married) 
'w5_mar_year', # year of Marriage (previously couple)
'w5_mar_month', # month of Marriage (previously couple)
# 'w5_status', # important (better than w5_complete)
#  'w5_duration',
#  'w5_source',
#  'w5x_qualified', # is subject qualified to take wave 5?
#  'w5x_marry', #NO -  married as of last contact
#  'w5x_last',# how many years ago was last contact
#  'w5x_year', # year of last contact
#  'w5x_civil', #NO -  does subject have civil union or domestic partnership prior to wave 5
#  'w5x_month', # month of last contact
#  'w5x_civyear', #NO  -  year last asked the civil union question
#  'w5x_civmonth',
# MARRIED
# 'w5_q1', # still married to partner?
# 'w5_q2', # are you currently living with partner?
# 'w5_sex_frequency', 
# 'w5_p_monogamy', # do you expect partner to be monogamous?
# 'w5_identity', # homosexual?
# 'w5_outness', # how many people aware you are gay
# 'w5_q3', #NO -  how did your marriage end?
# 'w5_q4', #NO - who watned divorce
# COUPLES
# 'w5_q5', # still in romantic relationship with partner?
# 'w5_q6', # currently living with partner?
# 'w5_q8', #  since last contact, have you gotten into domestic partnership?
# 'w5_civ_year',
# 'w5_civ_month',
# 'w5_q9', # how did relationship end?
# 'w5_q10', # who wanted to breakup?
# 'w2345_combo_breakup',
# 'responded_to_w234or5',
]].copy()

In [14]:
wv5_w_v2_columns = list(wv5_w_v2)
print(wv5_w_v2.shape)
wv5_w_v2

(4002, 13)


,caseid_new,caseid_wave,qualified?,w5_complete,w5_HCMST_interview_fin_yrmo,w5_broke_up,w5_q7,w5_mbtiming_month,w5_mbtiming_year,w5_nonmbtiming_year,w5_nonmbtiming_month,w5_mar_year,w5_mar_month
0,22526,22526_5,qualified,wave 5 completed,201412.0,still together,"no, did not marry [name]",NaN,NaN,NaN,NaN,NaN,NaN
1,23286,23286_5,qualified,wave 5 completed,201412.0,still together,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25495,25495_5,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,26315,26315_5,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27355,27355_5,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,27695,27695_5,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,28536,28536_5,qualified,wave 5 completed,201412.0,still together,"no, did not marry [name]",NaN,NaN,NaN,NaN,NaN,NaN
7,29584,29584_5,qualified,wave 5 completed,201412.0,still together,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,30393,30393_5,disqualified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,31456,31456_5,qualified,eligible but not completed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Wave 6

In [15]:
wv6_w_v1['qualified?'] = "qualified"
wv6_w_v1['survey_completed'] = "yes"

In [16]:
wv6_w_v2 = wv6_w_v1[[
'caseid_new',
'caseid_wave',
'qualified?',  # MANUALLY ADDED, EVERYOEN QUALIFIED
'survey_completed',
'w6_HCMST_yrmo', 
'w6_broke_up', # couple broke up in wave 6
'w6_got_married', # did you get marreid in the last year?    
'w6_mbtiming_Year',
'w6_mbtiming_Month',  
'w6_nonmbtiming_Year',
'w6_nonmbtiming_Month',
'w6_martiming_Year',
'w6_martiming_Month',
#  'xw6_year',
#  'xw6_month',
#  'xw6_last',
#  'xw6_marry',
#  'w6_still_mar',
#  'w6_live_together',
#  'w6_otherdate',
#  'w6_how_many',
#  'w6_otherdate_app',
#  'w6_how_many_app',
#  'w6_relationship_end_mar',
#  'w6_who_breakup',
#  'w6_partner_passaway_Year',
#  'w6_partner_passaway_Month',
#  'w6_otherdate_B',
#  'w6_how_many_B',
#  'w6_otherdate_app_B',
#  'w6_how_many_app_B',
#  'w6_still_rel',
#  'w6_live_together_C',

#  'w6_otherdate_C',
#  'w6_how_many_C',
#  'w6_otherdate_app_C',
#  'w6_how_many_app_C',
#  'w6_relationship_end_nonmar',
#  'w6_who_breakup_C',
#  'w6_partner_passaway_C_Year',
#  'w6_partner_passaway_C_Month',
#  'w6_otherdate_C2',
#  'w6_how_many_C2',
#  'w6_otherdate_app_C2',
#  'w6_how_many_app_C2',
# 'w6_relate_status',
#  'w6_met_through_friends',
#  'w6_met_through_family',
#  'w6_met_online',
#  'w6_met_through_as_coworkers',
#  'w6_met_online_dating',
#  'w6_met_bar_restaurant',
#  'w6_how_met_story_length',
#  'w6_combo_live_together',
#  'w6_otherdate_combo',
#  'w6_how_many_combo',
#  'w6_otherdate_app_combo',
#  'w6_how_many_app_combo',
]]

In [17]:
wv6_w_v2.head()

,caseid_new,caseid_wave,qualified?,survey_completed,w6_HCMST_yrmo,w6_broke_up,w6_got_married,w6_mbtiming_Year,w6_mbtiming_Month,w6_nonmbtiming_Year,w6_nonmbtiming_Month,w6_martiming_Year,w6_martiming_Month
0,22526,22526_6,qualified,yes,201707,still together,"No, did not marry PARTNER NAME",NaN,NaN,NaN,NaN,NaN,NaN
1,28536,28536_6,qualified,yes,201707,still together,"No, did not marry PARTNER NAME",NaN,NaN,NaN,NaN,NaN,NaN
2,29584,29584_6,qualified,yes,201707,still together,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,48225,48225_6,qualified,yes,201707,still together,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49731,49731_6,qualified,yes,201707,still together,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
print(f'wave2 table shape: {wv2_w_v2.shape}')
print(f'wave3 table shape: {wv3_w_v2.shape}')
print(f'wave4 table shape: {wv4_w_v2.shape}')
print(f'wave5 table shape: {wv5_w_v2.shape}')
print(f'wave6 table shape: {wv6_w_v2.shape}')

wave2 table shape: (3009, 7)
wave3 table shape: (3009, 11)
wave4 table shape: (4002, 16)
wave5 table shape: (4002, 13)
wave6 table shape: (640, 13)


# Making tables with uniform columns/values

In [19]:
# re-naming columns
#wave 2
wv2_w_v3 = wv2_w_v2.rename(columns={
    'w2_f1complete':'survey_completed',
    'w2_HCMST_interview_fin_yrmo':'followup_HCMST_yrmo',
    'w2_broke_up':'relationship_status',
    'w2_q7':'got_married?'
}).copy()

# wave 3
wv3_w_v3 = wv3_w_v2.rename(columns={
    'w3_complete':'survey_completed',
    'w3_HCMST_interview_fin_yrmo':'followup_HCMST_yrmo',
    'w3_broke_up':'relationship_status',
    'w3_q7':'got_married?',
    'w3_mbtiming_year':'breakup_year_married',
    'w3_mbtiming_month':'breakup_month_married',
    'w3_nonmbtiming_year':'breakup_year_nonmarried',
    'w3_nonmbtiming_month':'breakup_month_nonmarried'  
}).copy()

#wave 4
wv4_w_v3 = wv4_w_v2.rename(columns={
    'w4_complete':'survey_completed',
    'w4_HCMST_interview_fin_yrmo':'followup_HCMST_yrmo',
    'w4_broke_up':'relationship_status',
    'w4_q7':'got_married?',
    'w4_mbtiming_year':'breakup_year_married',
    'w4_mbtiming_month':'breakup_month_married',
    'w4_nonmbtiming_year':'breakup_year_nonmarried',
    'w4_nonmbtiming_month':'breakup_month_nonmarried',
    'w4_mar_year':'married_year',
    'w4_mar_month':'married_month',
    'relationship_quality_w4':'relationship_quality',
    'w4_attractive':'attractive_self', 
    'w4_attractive_partner':'attractive_partner'
}).copy()

#wave 5
wv5_w_v3 = wv5_w_v2.rename(columns={
    'w5_complete':'survey_completed',
    'w5_HCMST_interview_fin_yrmo':'followup_HCMST_yrmo',
    'w5_broke_up':'relationship_status',
    'w5_q7':'got_married?',
    'w5_mbtiming_year':'breakup_year_married',
    'w5_mbtiming_month':'breakup_month_married',
    'w5_nonmbtiming_year':'breakup_year_nonmarried',
    'w5_nonmbtiming_month':'breakup_month_nonmarried',
    'w5_mar_year':'married_year',
    'w5_mar_month':'married_month'
}).copy()

# wave 6
wv6_w_v3 = wv6_w_v2.rename(columns={
    'w6_HCMST_yrmo':'followup_HCMST_yrmo',
    'w6_broke_up':'relationship_status',
    'w6_got_married':'got_married?',
    'w6_mbtiming_Year':'breakup_year_married',
    'w6_mbtiming_Month':'breakup_month_married',
    'w6_nonmbtiming_Year':'breakup_year_nonmarried',
    'w6_nonmbtiming_Month':'breakup_month_nonmarried',
    'w6_martiming_Year':'married_year',
    'w6_martiming_Month':'married_month'
}).copy()

In [20]:
v3_list_of_dfs = [wv2_w_v3, wv3_w_v3, wv4_w_v3, wv5_w_v3, wv6_w_v3]

In [21]:
# Functions Combining Breakup Dates
def breakup_year(married,relationship):
    if married>0:
        return married
    elif relationship>0:
        return relationship
    else:
        return 

def breakup_month(married,relationship):
    if type(married)== str:
        return married
    elif type(relationship) == str:
        return relationship
    else:
        return 

In [26]:
# Combining Breakup Dates
#NOTE# wave 2 doesn't have breakup date info
# wave 3
wv3_w_v3['breakup_year_combo']= wv3_w_v3.apply(lambda x: 
                                        breakup_year(x['breakup_year_married'],
                                                     x['breakup_year_nonmarried']),
                                                     axis=1)
wv3_w_v3['breakup_month_combo']= wv3_w_v3.apply(lambda x: 
                                        breakup_month(x['breakup_month_married'],
                                                     x['breakup_month_nonmarried']),
                                                     axis=1)

# wave 4
wv4_w_v3['breakup_year_combo']= wv4_w_v3.apply(lambda x: 
                                        breakup_year(x['breakup_year_married'],
                                                     x['breakup_year_nonmarried']),
                                                     axis=1)
wv4_w_v3['breakup_month_combo']= wv4_w_v3.apply(lambda x: 
                                        breakup_month(x['breakup_month_married'],
                                                     x['breakup_month_nonmarried']),
                                                     axis=1)

# wave 5
wv5_w_v3['breakup_year_combo']= wv5_w_v3.apply(lambda x: 
                                        breakup_year(x['breakup_year_married'],
                                                     x['breakup_year_nonmarried']),
                                                     axis=1)
wv5_w_v3['breakup_month_combo']= wv5_w_v3.apply(lambda x: 
                                        breakup_month(x['breakup_month_married'],
                                                     x['breakup_month_nonmarried']),
                                                     axis=1)

# wave 6
wv6_w_v3['breakup_year_combo']= wv6_w_v3.apply(lambda x: 
                                        breakup_year(x['breakup_year_married'],
                                                     x['breakup_year_nonmarried']),
                                                     axis=1)
wv6_w_v3['breakup_month_combo']= wv6_w_v3.apply(lambda x: 
                                        breakup_month(x['breakup_month_married'],
                                                     x['breakup_month_nonmarried']),
                                                     axis=1)


In [27]:
# function to make survey_completed answers uniform
def survey_completed(survey_completed):
    if survey_completed in ["completed followup survey" , "Yes" , "yes" , "wave 5 completed"]:
        return "yes"
    elif survey_completed in ["did not complete followup survey" , "No" , "no" , "eligible but not completed"]:
        return "no"
    else:
        return

In [28]:
for y in v3_list_of_dfs:
    y["survey_completed_new"] = y.apply(lambda x:
                                   survey_completed(x['survey_completed']),
                                   axis=1)

In [29]:
def get_married(answer):
    if answer in ["no, did not marry [partner]" , "no, did not marry [xNameP]" , "No, did not marry [xnamep]" , 
                  "no, did not marry [name]", "No, did not marry PARTNER NAME"]:
        return "no"
    elif answer in ["yes, married [partner]" , "yes, married [xNameP]" , "Yes, married [xnamep]" ,
                    "yes, married [name]","Yes, married PARTNER NAME"]:
        return "yes"
    else:
        return

In [30]:
for y in v3_list_of_dfs:
    y["got_married?_new"] = y.apply(lambda x:
                                   get_married(x['got_married?']),
                                   axis=1)

# v4 tables (only new columns)

In [41]:
wv2_w_v4 = wv2_w_v3[['caseid_new',
 'caseid_wave',
 'qualified?',
#  'survey_completed',
 'followup_HCMST_yrmo',
 'relationship_status',
#  'got_married?',
 'survey_completed_new',
 'got_married?_new']].copy()

In [162]:
wv2_w_v4

,caseid_new,caseid_wave,qualified?,followup_HCMST_yrmo,relationship_status,survey_completed_new,got_married?_new
0,22526,22526_2,qualified,201003.0,still together,yes,no
1,23286,23286_2,qualified,201003.0,still together,yes,None
2,26315,26315_2,qualified,201003.0,broke up,yes,None
3,28536,28536_2,qualified,201003.0,still together,yes,no
4,29584,29584_2,qualified,201003.0,still together,yes,None
5,31456,31456_2,qualified,NaN,NaN,no,None
6,32656,32656_2,qualified,NaN,NaN,no,None
7,33536,33536_2,qualified,201003.0,still together,yes,no
8,34341,34341_2,qualified,201003.0,still together,yes,None
9,35653,35653_2,qualified,201003.0,broke up,yes,None


In [58]:
wv3_w_v4 = wv3_w_v3[['caseid_new',
 'caseid_wave',
 'qualified?',
 'survey_completed_new',                     
#  'survey_completed',
 'followup_HCMST_yrmo',
 'relationship_status',
#  'got_married?',
#  'breakup_year_married',
#  'breakup_month_married',
#  'breakup_year_nonmarried',
#  'breakup_month_nonmarried',
 'breakup_year_combo',
 'breakup_month_combo',
 'got_married?_new']].copy()

In [59]:
wv4_w_v4 = wv4_w_v3[['caseid_new',
 'caseid_wave',
 'qualified?',
 'survey_completed_new',                     
#  'survey_completed',
 'followup_HCMST_yrmo',
 'relationship_status',
#  'got_married?',
#  'breakup_year_married',
#  'breakup_month_married',
#  'breakup_year_nonmarried',
#  'breakup_month_nonmarried',
 'married_year',
 'married_month',
 'relationship_quality',
 'attractive_self',
 'attractive_partner',
 'breakup_year_combo',
 'breakup_month_combo',
 'got_married?_new']].copy()

In [60]:
wv5_w_v4 = wv5_w_v3[['caseid_new',
 'caseid_wave',
 'qualified?',
 'survey_completed_new',
#  'survey_completed',
 'followup_HCMST_yrmo',
 'relationship_status',
#  'got_married?',
#  'breakup_month_married',
#  'breakup_year_married',
#  'breakup_year_nonmarried',
#  'breakup_month_nonmarried',
 'married_year',
 'married_month',
 'breakup_year_combo',
 'breakup_month_combo',
 'got_married?_new']].copy()

In [61]:
wv6_w_v4 = wv6_w_v3[['caseid_new',
'caseid_wave',
'qualified?',
'survey_completed_new',
#  'survey_completed',
'followup_HCMST_yrmo',
'relationship_status',
#  'got_married?',
#  'breakup_year_married',
#  'breakup_month_married',
#  'breakup_year_nonmarried',
#  'breakup_month_nonmarried',
'married_year',
'married_month',
'breakup_year_combo',
'breakup_month_combo',
'got_married?_new']].copy()

In [76]:
wv6_w_v4

,caseid_new,caseid_wave,qualified?,survey_completed_new,followup_HCMST_yrmo,relationship_status,married_year,married_month,breakup_year_combo,breakup_month_combo,got_married?_new
0,22526,22526_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,no
1,28536,28536_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,no
2,29584,29584_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,None
3,48225,48225_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,None
4,49731,49731_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,None
5,58295,58295_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,None
6,64331,64331_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,None
7,68546,68546_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,None
8,83431,83431_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,None
9,103545,103545_6,qualified,yes,201707,still together,NaN,NaN,NaN,None,no


In [62]:
for x in v3_list_of_dfs:
    print(x["qualified?"].value_counts())
    print("----")

qualified    3009
Name: qualified?, dtype: int64
----
qualified       2689
disqualified     320
Name: qualified?, dtype: int64
----
qualified       2557
disqualified    1445
Name: qualified?, dtype: int64
----
qualified       2444
disqualified    1558
Name: qualified?, dtype: int64
----
qualified    640
Name: qualified?, dtype: int64
----


In [63]:
v4_list_of_dfs = [wv2_w_v4, wv3_w_v4, wv4_w_v4, wv5_w_v4, wv6_w_v4]

In [132]:
for x in v4_list_of_dfs:
    print(x['got_married?_new'].value_counts())
    print("----")

no     605
yes     45
Name: got_married?_new, dtype: int64
----
no     436
yes     36
Name: got_married?_new, dtype: int64
----
no     279
yes     33
Name: got_married?_new, dtype: int64
----
no     171
yes     34
Name: got_married?_new, dtype: int64
----
no     77
yes    12
Name: got_married?_new, dtype: int64
----


In [180]:
xaxis = ['wave 2', 'wave 3', 'wave 4', 'wave 5', 'wave 6']

In [85]:
import plotly.graph_objects as go
import numpy as np

In [176]:
still_together = []
broke_up = []
partner_deceased = []
got_married = []
number_of_participants = []
qualified = []
disqualified = []

for x in v4_list_of_dfs:
    still_together.append(x['relationship_status'].value_counts()[0])
    broke_up.append(x['relationship_status'].value_counts()[1])
    partner_deceased.append(x['relationship_status'].value_counts()[2])
    got_married.append(x['got_married?_new'].value_counts()[1])
    number_of_participants.append(x['survey_completed_new'].value_counts()[0])
    qualified.append(x['qualified?'].value_counts()[0])
    
for x in v4_list_of_dfs:
    if len(x['qualified?'].value_counts())>1:
        disqualified.append(x['qualified?'].value_counts()[1])   
    else:
        disqualified.append(0)
        
print(still_together)
print(broke_up)
print(partner_deceased)
print(number_of_participants)
print(qualified)
print(disqualified)

[2231, 1828, 1425, 1066, 608]
[248, 112, 95, 48, 24]
[41, 20, 16, 13, 8]
[2520, 1960, 1536, 1317, 640]
[3009, 2689, 2557, 2444, 640]
[0, 320, 1445, 1558, 0]


In [136]:
def cumulative_sum(x):
    l=len(x)
    s=[]
    for i in range(1,l+1):
        s.append(sum(x[:i]))
    return s


print(cumulative_sum(broke_up))
broke_up_cumulative = cumulative_sum(broke_up)
deceased_cumulative = cumulative_sum(partner_deceased)
got_married_cumulative = cumulative_sum(got_married)

[248, 360, 455, 503, 527]


In [177]:
summaryStats = pd.DataFrame(list(zip(x, number_of_participants, still_together, broke_up, broke_up_cumulative, partner_deceased, 
                                     deceased_cumulative, got_married,got_married_cumulative, qualified, disqualified)),
                            columns = ["wave","participants","still_together","broke_up","broke_up_cumulative","partner_deceased",
                                      "partner_deceased_cumulative","got_married","got_married_cumulative", "qualified","disqualified"])

summaryStats = summaryStats.set_index("wave")
# summaryStats = summaryStats.transpose()
summaryStats

,participants,still_together,broke_up,broke_up_cumulative,partner_deceased,partner_deceased_cumulative,got_married,got_married_cumulative,qualified,disqualified
wave,,,,,,,,,,
caseid_new,2520,2231,248,248,41,41,45,45,3009,0
caseid_wave,1960,1828,112,360,20,61,36,81,2689,320
qualified?,1536,1425,95,455,16,77,33,114,2557,1445
survey_completed_new,1317,1066,48,503,13,90,34,148,2444,1558
followup_HCMST_yrmo,640,608,24,527,8,98,12,160,640,0


In [179]:
summaryStats['%_still_together_ofParticipant'] = summaryStats.apply(lambda x : x["still_together"]/x["participants"]*100,axis=1)
summaryStats['%_still_together_ofQualified'] = summaryStats.apply(lambda x : x["still_together"]/x["qualified"]*100,axis=1)
summaryStats

,participants,still_together,broke_up,broke_up_cumulative,partner_deceased,partner_deceased_cumulative,got_married,got_married_cumulative,qualified,disqualified,%_still_together_ofParticipant,%_still_together_ofQualified
wave,,,,,,,,,,,,
caseid_new,2520,2231,248,248,41,41,45,45,3009,0,88.531746,74.144234
caseid_wave,1960,1828,112,360,20,61,36,81,2689,320,93.265306,67.980662
qualified?,1536,1425,95,455,16,77,33,114,2557,1445,92.773438,55.729370
survey_completed_new,1317,1066,48,503,13,90,34,148,2444,1558,80.941534,43.617021
followup_HCMST_yrmo,640,608,24,527,8,98,12,160,640,0,95.000000,95.000000


In [184]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=xaxis, y=qualified,
                    mode='lines+markers',
                    name='qualified',
                    line=dict(color='black', width=3)))
fig.add_trace(go.Scatter(x=xaxis, y=number_of_participants,
                    mode='lines+markers',
                    name='participated',
                    line=dict(color='black', width=3, 
                             dash = "dash")))
fig.add_trace(go.Scatter(x=xaxis, y=still_together,
                    mode='lines+markers',
                    name='still together (and participated)'))
fig.add_trace(go.Scatter(x=xaxis, y=broke_up,
                    mode='markers',
                    name='broke up',
                    line=dict(color='firebrick', width=2)))
fig.add_trace(go.Scatter(x=xaxis, y=broke_up_cumulative,
                    mode='lines+markers',
                    name='broke up (cumulative)',
                    line=dict(color='firebrick', width=2)))
fig.add_trace(go.Scatter(x=xaxis, y=partner_deceased,
                    mode='markers',
                    name='partner deceased',
                    line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=xaxis, y=deceased_cumulative,
                    mode='lines+markers',
                    name='partner deceased (cumulative)',
                    line=dict(color='green', width=2)))
fig.update_layout(title='Relationship Status Summary',
                   xaxis_title='Wave',
                   yaxis_title='Number of People')
fig.show()

# Export to CSV

In [65]:
wv2_w_v4.to_csv("CSV_files/tables_W/wv2_w_v4.csv", index=False, header=True)
wv3_w_v4.to_csv("CSV_files/tables_W/wv3_w_v4.csv", index=False, header=True)
wv4_w_v4.to_csv("CSV_files/tables_W/wv4_w_v4.csv", index=False, header=True)
wv5_w_v4.to_csv("CSV_files/tables_W/wv5_w_v4.csv", index=False, header=True)
wv6_w_v4.to_csv("CSV_files/tables_W/wv6_w_v4.csv", index=False, header=True)

# Sqlite


In [66]:
connection_string = "Relationship_survey.sqlite"
engine2 = create_engine(f'sqlite:///{connection_string}')
engine2.table_names() # now it's empty

['relationship_qs',
 'wv1_pp',
 'wv2_pp',
 'wv2_w',
 'wv3_pp',
 'wv3_w',
 'wv4_pp',
 'wv4_w',
 'wv5_pp',
 'wv5_w',
 'wv6_pp',
 'wv6_w']

In [67]:
wv2_w_v4.to_sql(name="wv2_w_2", con=engine2, if_exists='replace', index=True)
wv3_w_v4.to_sql(name="wv3_w_2", con=engine2, if_exists='replace', index=True)
wv4_w_v4.to_sql(name="wv4_w_2", con=engine2, if_exists='replace', index=True)
wv5_w_v4.to_sql(name="wv5_w_2", con=engine2, if_exists='replace', index=True)
wv6_w_v4.to_sql(name="wv6_w_2", con=engine2, if_exists='replace', index=True)
#wv2_w.to_sql(name="wv1_pp", con=engine2, if_exists='replace', index=True)
engine2.table_names()

['relationship_qs',
 'wv1_pp',
 'wv2_pp',
 'wv2_w',
 'wv2_w_2',
 'wv3_pp',
 'wv3_w',
 'wv3_w_2',
 'wv4_pp',
 'wv4_w',
 'wv4_w_2',
 'wv5_pp',
 'wv5_w',
 'wv5_w_2',
 'wv6_pp',
 'wv6_w',
 'wv6_w_2']

In [69]:
# check if data has been stored properly 

sqlite_wv2_w=engine2.execute("SELECT * FROM wv2_w_2").fetchall()
for elem in sqlite_wv2_w[0:5]:
    print(elem)
    print("\n")

(0, 22526, '22526_2', 'qualified', 201003.0, 'still together', 'yes', 'no')


(1, 23286, '23286_2', 'qualified', 201003.0, 'still together', 'yes', None)


(2, 26315, '26315_2', 'qualified', 201003.0, 'broke up', 'yes', None)


(3, 28536, '28536_2', 'qualified', 201003.0, 'still together', 'yes', 'no')


(4, 29584, '29584_2', 'qualified', 201003.0, 'still together', 'yes', None)


